In [51]:
using LinearAlgebra, ToeplitzMatrices, Random, IterativeSolvers, FunctionOperators, EllipsisNotation, Printf

In [12]:
d = 10
v = rand(d)
Xᴳᵀ = v * v'  # Ground Truth matrix
@show size(Xᴳᵀ)
@show rank(Xᴳᵀ)

# mask that erases 5 elements:
num_of_points_to_erase = 5
Φᴹ = reshape(shuffle!([fill(0, num_of_points_to_erase)...,
            fill(1, d*d - num_of_points_to_erase)...]), d, d)
bit_mask = Φᴹ .== 1
m = Int(sum(Φᴹ))
Φ = FunctionOperator{Float64}(name = "Φ", inDims = (d, d), outDims = (m,),
    forw = (b,x) -> b .= x[bit_mask], backw = (b,x) -> begin b[bit_mask] .= x; b; end)

y = Φ * Xᴳᵀ
@show size(y)
@show size(Φ' * y)
@show rank(Φ' * y);

size(Xᴳᵀ) = (10, 10)
rank(Xᴳᵀ) = 1
size(y) = (95,)
size(Φ' * y) = (10, 10)
rank(Φ' * y) = 5


### The algorithm itself

In [56]:
function HM_IRLS(
        Xᴳᵀ::AbstractArray,                   # ground truth for MSE evaluation
        y::AbstractArray,                     # under-sampled data
        Φ::FunctionOperator;                  # sampling operator
        shape::NTuple = size(Xᴳᵀ),            # size of output matrix
        r̃::Int = 0,                           # rank estimate of solution
        maxIter::Int = 3,                     # number of CG iteration steps
        N::Int = 10,                          # number of iterations
        verbose::Bool = false)                # print rank and loss value in each iteration
    
    dType = eltype(y)
    d₁, d₂ = shape
    r̃ == 0 && (r̃ = rank(Xᴳᵀ))
    
    ϵᵏ = Inf
    Xᵏ = Φ' * y
    
    for k in 1:N
"""
    2. Find best rank-(r̃ + 1) approximation of Xᵏ to obtain
        𝒯ᵣ(Xᵏ) = Uᵏ * diag(σᵢᵏ)ᵢ₌₁ʳ * Vᵏ' and σᵣ₊₁ᵏ 
"""
        F = svd(Xᵏ)
        Uᵏ, σ, Vᵏ = F.U[:, 1:r̃], F.S, F.V[:, 1:r̃]
        
"""     update smoothing:                                 (2.91) """
        ϵᵏ = min(ϵᵏ, σ[r̃+1])
        
        r, n, s, e = rank(Xᵏ), norm₂(Xᴳᵀ - Xᵏ), σ[1], ϵᵏ
        n, s, e = @sprintf("%.3f", n), @sprintf("%.3f", s), @sprintf("%.3f", e)
        println("k = $(k-1),\trank(Xᵏ) = $r,\t‖Xᴳᵀ - Xᵏ‖₂ = $n, σ₁ = $s, ϵᵏ = $e")
        
"""
    3. Update Wᵏ as in (2.57), using parameters ϵ = ϵᵏ and p in (2.58) and (2.59), and the
        information Uᵏ , Vᵏ and σ₁ᵏ, ..., σᵣ₊₁ᵏ from item 2.

        (Lines below are based on Remark 2.3.2, the special case for p = 0)
"""
        # Hᵏ = [1 / (max(σ[i], ϵᵏ) * max(σ[j], ϵᵏ))  for i in 1:r̃+1, j in 1:r̃+1]
        #Wᵏ = FunctionOperator{dType}(name = "Wᵏ", inDims = (d₁, d₂), outDims = (d₁, d₂),
        #    forw = Z -> Uᵏ * (Hᵏ .* (Uᵏ' * Z * Vᵏ)) * Vᵏ')
        
"""
    1. Use a conjugate gradient method to solve linearly constrained quadratic program
         Xᵏ = arg minₓ ⟨X,Wᵏ⁻¹(X)⟩ s.t. Φ(X) = y         (2.90)
"""
        
        Hᵏᵤᵥ = [1 / (max(σ[i], ϵᵏ) * max(σ[j], ϵᵏ))  for i in 1:r̃, j in 1:r̃] # the upper-left (r × r) block of (d₁ × d₂) Hᵏ matrix
        dHᵏ = reshape([1 / (max(σ[r̃+1], ϵᵏ) * max(σ[j], ϵᵏ))  for j in 1:r̃], :, 1) # the first column of Hᵏᵤᵥ⟂
        Pᵏ = FunctionOperator{dType}(name="Pᵏ", inDims = (r̃*(r̃+d₁+d₂),), outDims = (d₁, d₂),
            forw = γ -> begin
                    γ = reshape(γ, r̃, r̃+d₁+d₂)
                    γ₁, γ₂, γ₃ = γ[:,1:r̃], γ[:,r̃+1:r̃+d₁], γ[:,r̃+d₁+1:r̃+d₁+d₂]
                    (Uᵏ * γ₁ + γ₂') * Vᵏ' + Uᵏ * γ₃
                end,
            backw = Φᵃy -> begin
                    γ₁ = Uᵏ' * Φᵃy * Vᵏ
                    γ₂ = (I - Uᵏ*Uᵏ') * Φᵃy * Vᵏ
                    γ₃ = Uᵏ' * Φᵃy * (I - Vᵏ*Vᵏ')
                    vec(hcat(γ₁, γ₂', γ₃)) # is it ok to transpose γ₂ ??
                end)
        b = Pᵏ' * Φ' * y
        𝒟⁻¹ = Diagonal(vec(hcat(Hᵏᵤᵥ, repeat(dHᵏ, outer=(1, d₁)), repeat(dHᵏ, outer=(1, d₂)))))
        CG_op = FunctionOperator{dType}(name = "CG_op", inDims = (r̃*(r̃+d₁+d₂),), outDims = (r̃*(r̃+d₁+d₂),),
            forw = γ ->  begin
                    #println(size((ϵᵏ^2 * I / (𝒟⁻¹ - ϵᵏ^2 * I)) * γ))
                    #println(size(Φ * Pᵏ * γ))
                    #println(size(Pᵏ' * Φ' * Φ * Pᵏ * γ))
                    (ϵᵏ^2 * I / (𝒟⁻¹ - ϵᵏ^2 * I)) * γ + Pᵏ' * Φ' * Φ * Pᵏ * γ
                end)
        γᵏ = cg(CG_op, b, maxiter = maxIter) # 2.167
        rᵏ = y - Φ * Pᵏ * γᵏ
        γᵏ_tilde = (𝒟⁻¹ / (𝒟⁻¹ - ϵᵏ^2 * I)) * γᵏ - Pᵏ' * Φ' * rᵏ
        Xᵏ = Φ' * rᵏ + Pᵏ * γᵏ_tilde   # 2.168
    end
    
    println("k = $(k-1),\trank(Xᵏ) = $r,\t‖Xᴳᵀ - Xᵏ‖₂ = $n, σ₁ = $s, ϵᵏ = $e")
    
    Xᵏ
end

HM_IRLS (generic function with 1 method)

In [60]:
@time HM_IRLS(Xᴳᵀ, y, Φ, maxIter = 100, N = 20);

k = 0,	rank(Xᵏ) = 100,	‖Xᴳᵀ - Xᵏ‖₂ = 197.185, σ₁ = 35.775, ϵᵏ = 18.771
k = 1,	rank(Xᵏ) = 100,	‖Xᴳᵀ - Xᵏ‖₂ = 225.056, σ₁ = 135.500, ϵᵏ = 18.771
k = 2,	rank(Xᵏ) = 100,	‖Xᴳᵀ - Xᵏ‖₂ = 191.552, σ₁ = 45.714, ϵᵏ = 18.771
k = 3,	rank(Xᵏ) = 100,	‖Xᴳᵀ - Xᵏ‖₂ = 226.305, σ₁ = 106.121, ϵᵏ = 18.771
k = 4,	rank(Xᵏ) = 100,	‖Xᴳᵀ - Xᵏ‖₂ = 190.294, σ₁ = 46.880, ϵᵏ = 18.771
k = 5,	rank(Xᵏ) = 100,	‖Xᴳᵀ - Xᵏ‖₂ = 226.261, σ₁ = 95.904, ϵᵏ = 18.771
k = 6,	rank(Xᵏ) = 100,	‖Xᴳᵀ - Xᵏ‖₂ = 190.149, σ₁ = 47.067, ϵᵏ = 18.771
k = 7,	rank(Xᵏ) = 100,	‖Xᴳᵀ - Xᵏ‖₂ = 226.096, σ₁ = 92.236, ϵᵏ = 18.771
k = 8,	rank(Xᵏ) = 100,	‖Xᴳᵀ - Xᵏ‖₂ = 189.924, σ₁ = 47.144, ϵᵏ = 18.771
k = 9,	rank(Xᵏ) = 100,	‖Xᴳᵀ - Xᵏ‖₂ = 226.059, σ₁ = 91.270, ϵᵏ = 18.771
k = 10,	rank(Xᵏ) = 100,	‖Xᴳᵀ - Xᵏ‖₂ = 189.931, σ₁ = 47.172, ϵᵏ = 18.771
k = 11,	rank(Xᵏ) = 100,	‖Xᴳᵀ - Xᵏ‖₂ = 226.013, σ₁ = 91.011, ϵᵏ = 18.771
k = 12,	rank(Xᵏ) = 100,	‖Xᴳᵀ - Xᵏ‖₂ = 189.885, σ₁ = 47.148, ϵᵏ = 18.771
k = 13,	rank(Xᵏ) = 100,	‖Xᴳᵀ - Xᵏ‖₂ = 226.011, σ₁ = 90.976, ϵᵏ = 18.771


In [83]:
function HM_IRLS(
        Xᴳᵀ::AbstractArray,                   # ground truth for MSE evaluation
        y::AbstractArray,                     # under-sampled data
        Φ::FunctionOperator;                  # sampling operator
        shape::NTuple = size(Xᴳᵀ),            # size of output matrix
        r̃::Int = 0,                           # rank estimate of solution
        maxIter::Int = 3,                     # number of CG iteration steps
        N::Int = 10,                          # number of iterations
        verbose::Bool = false)                # print rank and loss value in each iteration
    
    dType = eltype(y)
    d₁, d₂ = shape
    r̃ == 0 && (r̃ = rank(Xᴳᵀ))
    
    X⁰ = Φ' * y
    Wᵏ = I
    b = [y; zeros(dType, (d₁, d₂))]
    ϵᵏ = Inf
    Xᵏ = copy(X⁰)
    
    println("k = 0,\trank(Xᵏ) = $(rank(Xᵏ)),\t‖Xᴳᵀ - Xᵏ‖₂ = $(norm(Xᴳᵀ - Xᵏ))")
    
    for k in 1:N
        
"""
    1. Use a conjugate gradient method to solve linearly constrained quadratic program
         Xᵏ = arg minₓ ⟨X,Wᵏ⁻¹(X)⟩ s.t. Φ(X) = y         (2.90)
"""
        CG_op = FunctionOperator{dType}(name = "CG_op", inDims = (d₁*d₂*2,), outDims = (d₁*d₂*2,),
            forw = x_ext -> begin
                x = reshape(x_ext, 2, d₁, d₂)[1, ..]
                vec([Φ' * Φ * x; Wᵏ * x])
            end)
        Xᵏ_ext = [Xᵏ; zeros(dType, (d₁, d₂))]
        bicgstabl!(vec(Xᵏ_ext), CG_op, vec(b), #Pl = ,
            max_mv_products = maxIter) # stabilized biconjugate gradients
        Xᵏ_ext[1:end÷2, :]
        Xᵏ = Xᵏ_ext[1:end÷2, :] # crop the the part we need
        
"""
    2. Find best rank-(r̃ + 1) approximation of Xᵏ to obtain
        𝒯ᵣ(Xᵏ) = Uᵏ * diag(σᵢᵏ)ᵢ₌₁ʳ * Vᵏ' and σᵣ₊₁ᵏ 
"""
        F = svd(Xᵏ)
        Uᵏ, σ, Vᵏ = F.U[:, 1:r̃+1], F.S, F.V[:, 1:r̃+1]
        println("σ₁: ", σ[1])
"""     update smoothing:                                 (2.91) """
        ϵᵏ = min(ϵᵏ, σ[r̃+1])
        println("ϵᵏ: ", ϵᵏ)
        
"""
    3. Update Wᵏ as in (2.57), using parameters ϵ = ϵᵏ and p in (2.58) and (2.59), and the
        information Uᵏ , Vᵏ and σ₁ᵏ, ..., σᵣ₊₁ᵏ from item 2.

        (Lines below are based on Remark 2.3.2, the special case for p = 0)
"""
        Hᵏ = [1 / (max(σ[i], ϵᵏ) * max(σ[j], ϵᵏ))  for i in 1:r̃+1, j in 1:r̃+1] # 
        Wᵏ = FunctionOperator{dType}(name = "Wᵏ", inDims = (d₁, d₂), outDims = (d₁, d₂),
            forw = Z -> Uᵏ * (Hᵏ .* (Uᵏ' * Z * Vᵏ)) * Vᵏ')
        
        println("k = $k,\trank(Xᵏ) = $(rank(Xᵏ)),\t‖Xᴳᵀ - Xᵏ‖₂ = $(norm₂(Xᴳᵀ - Xᵏ))")
    end
    
    Xᵏ
end

HM_IRLS (generic function with 1 method)

In [85]:
@time HM_IRLS(Xᴳᵀ, y, Φ, maxIter = 200, N = 10);

k = 0,	rank(Xᵏ) = 100,	‖Xᴳᵀ - Xᵏ‖₂ = 249.87729971831368
σ₁: 8.434948101987105e15
ϵᵏ: 4.5078289462956025e14
k = 1,	rank(Xᵏ) = 99,	‖Xᴳᵀ - Xᵏ‖₂ = 8.43494810198711e15
σ₁: 5.337565515140542e45
ϵᵏ: 4.5078289462956025e14
k = 2,	rank(Xᵏ) = 25,	‖Xᴳᵀ - Xᵏ‖₂ = 5.337565515140541e45
σ₁: 7.731340991909634e114
ϵᵏ: 4.5078289462956025e14
k = 3,	rank(Xᵏ) = 8,	‖Xᴳᵀ - Xᵏ‖₂ = 7.731340991909632e114


SingularException: SingularException(2)

### Some helper functions

In [2]:
import Base.size
function Base.size(FO::FunctionOperator, d::Int)
    @assert d in [1, 2]
    prod(d == 1 ? FO.outDims : FO.inDims)
end

In [3]:
norm₂ = (A) -> svdvals(A)[1]

#3 (generic function with 1 method)

In [4]:
# This function randomly samples a $(d₁ \times d₂)$ sparse matrix with ones at $m$ randomly chosen
# coordinates (uniform without replacement). The output matrix has at least $r$ non-zero entries
# in each row and each column, where $r$ is a specified positive integer. The number of ones in the
# output matrix is exactly $m$.
function generateΦ(d₁, d₂, r, m)
    @assert max(d₁, d₂) * r ≤ m
    @assert m ≤ d₁ * d₂
    @assert r ≤ d₁
    @assert r ≤ d₂
    
    # generate a square matrix where each row and each column has exactly r ones
    initial = Circulant([fill(1, r)..., fill(0, min(d₁, d₂) - r)...])
    
    # Extend that matrix to a d₁×d₂ matrix where each row and each column has at least r ones
    # That is accomplished by repeating the "initial" matrix and then cropping
    if d₁ < d₂
        M = repeat(initial, outer = (1, ceil(Int, d₂ / d₁)))
    elseif d₁ > d₂
        M = repeat(initial, outer = (ceil(Int, d₁ / d₂), 1))
    else
        M = initial
    end
    M = M[1:d₁, 1:d₂]
    
    # Randomly switch zeros to ones until exactly m number of ones are in the matrix
    zero_places = findall(M .== 0)
    number_of_missing_ones = m - (d₁*d₂ - length(zero_places))
    number_of_missing_ones > 0 && (M[shuffle(zero_places)[1:number_of_missing_ones]] .= 1)
    
    # Then randomize matrix by permutating rows and columns a couple times
    for i in 1:10
        M .= M[shuffle(1:end), :] # shuffle rows
        M .= M[:, shuffle(1:end)] # shuffle columns
    end
    
    M
end

generateΦ (generic function with 1 method)

In [5]:
function maskToMatrix(Φᴹ)
    m = convert(Int, sum(Φᴹ))
    d₁, d₂ = size(Φᴹ)

    Φ = zeros(m, length(Φᴹ))
    non_zero_places = findall(vec(Φᴹ) .== 1)
    for i in 1:m
        Φ[i, non_zero_places[i]] = 1
    end
end

maskToMatrix (generic function with 1 method)

### Generate data

#### That's how Chirstian generated the data to compare algorithms:

In [6]:
d₁, d₂, r = 100, 100, 7
df_LR = r * (d₁ + d₂ - r) # Number of degrees of freedom of the setting
m = floor(Int, min(1.05 * df_LR, d₁ * d₂))

dType = ComplexF64
U, S, V = randn(dType, d₁, r), Diagonal(randn(r)), randn(dType, d₂, r)
Xᴳᵀ = U * S * V' # Ground Truth matrix

@show size(Xᴳᵀ)
@show rank(Xᴳᵀ);

Φᴹ = generateΦ(d₁, d₂, r, m)
Φ = FunctionOperator{dType}(name = "Φ", inDims = (d₁, d₂), outDims = (d₁, d₂),
    forw = (b,x) -> b .= Φᴹ .* x, backw = (b,x) -> b .= x)
y = Φ * Xᴳᵀ
@show rank(y);

size(Xᴳᵀ) = (100, 100)
rank(Xᴳᵀ) = 7
rank(y) = 100


In [7]:
Φᴹ .* Xᴳᵀ == Φ * Xᴳᵀ

true

In [24]:
@time HM_IRLS(Xᴳᵀ, y, Φ, maxIter = 200, N = 10);

k = 0,	rank(Xᵏ) = 100,	‖Xᴳᵀ - Xᵏ‖₂ = 229.55303757693594
5.283342883762677e18
27.703482115888328
k = 1,	rank(Xᵏ) = 99,	‖Xᴳᵀ - Xᵏ‖₂ = 9.116281210837809e18
5.630430334974542e71
27.703482115888328
k = 2,	rank(Xᵏ) = 7,	‖Xᴳᵀ - Xᵏ‖₂ = 7.90630929664771e71
 ** On entry to DLASCLS parameter number  4 had an illegal value
 ** On entry to DLASCLS parameter number  4 had an illegal value


LAPACKException: LAPACKException(23)

#### An easy problem:

In [103]:
d = 10
v = rand(d)
Xᴳᵀ = v * v'  # Ground Truth matrix
@show size(Xᴳᵀ)
@show rank(Xᴳᵀ)

# mask that erases 5 elements:
num_of_points_to_erase = 5
Φᴹ = reshape(shuffle!([fill(0, num_of_points_to_erase)...,
            fill(1, d*d - num_of_points_to_erase)...]), d, d)
Φ = FunctionOperator{Float64}(name = "Φ", inDims = (d, d), outDims = (d, d),
    forw = (b,x) -> b .= Φᴹ .* x, backw = (b,x) -> b .= x)

y = Φ * Xᴳᵀ
@show rank(y);

size(Xᴳᵀ) = (10, 10)
rank(Xᴳᵀ) = 1
rank(y) = 4


In [104]:
@time HM_IRLS(Xᴳᵀ, y, Φ, N = 10);

k = 0,	rank(Xᵏ) = 4,	‖Xᴳᵀ - Xᵏ‖₂ = 0.55469945286125
k = 1,	rank(Xᵏ) = 4,	‖Xᴳᵀ - Xᵏ‖₂ = 5.020881565879722
k = 2,	rank(Xᵏ) = 4,	‖Xᴳᵀ - Xᵏ‖₂ = 5.030105996305296
k = 3,	rank(Xᵏ) = 4,	‖Xᴳᵀ - Xᵏ‖₂ = 6.3359699325930965
k = 4,	rank(Xᵏ) = 4,	‖Xᴳᵀ - Xᵏ‖₂ = 77.82130976070576
k = 5,	rank(Xᵏ) = 4,	‖Xᴳᵀ - Xᵏ‖₂ = 133.4250873941241
k = 6,	rank(Xᵏ) = 4,	‖Xᴳᵀ - Xᵏ‖₂ = 183.80576126779502
k = 7,	rank(Xᵏ) = 4,	‖Xᴳᵀ - Xᵏ‖₂ = 254.0177948525734
k = 8,	rank(Xᵏ) = 4,	‖Xᴳᵀ - Xᵏ‖₂ = 293.25797761437633
k = 9,	rank(Xᵏ) = 4,	‖Xᴳᵀ - Xᵏ‖₂ = 354.2763504623122
k = 10,	rank(Xᵏ) = 4,	‖Xᴳᵀ - Xᵏ‖₂ = 413.53799784140466
  0.043236 seconds (34.08 k allocations: 2.203 MiB)
